# Pull An Individual Project's Polygons from TM API and Convert to a GeoJSON

In [1]:
import yaml
import pandas as pd
from tm_api_utils import pull_tm_api_data, patch_tm_api_data
from tqdm import tqdm
import json
import sys
from datetime import datetime
sys.path.append('../src/')
import api_utils as api
import process_tm_api_results as clean
import geospatial_utils_NEW as geo

### Set file paths

In [2]:
# Today's date
today = datetime.today().strftime('%Y-%m-%d') # Check computer date before running

# Set project name and ID
proj_name = 'justdiggit'
proj_id = ['8cbda7fc-aa17-444b-8425-7047fade708c']

# Set file input/output paths
approved_projects_file = '../projects_all_approved_202502211226.csv' # List of approved projects (infile)
tm_api_pull_results_file = f'../data/individual_projects/tm_api_response_prod_{proj_name}_{today}.json' # Save a JSON file that stores the results of the TM API pull; read it back in to clean the results (outfile, infile)
polygon_features_file = f'../data/individual_projects/tm_api_{proj_name}_{today}.csv' # Save the cleaned polygon features csv in the terrafund-portfolio-analyses repo (outfile)
geojson_output = f'../data/individual_projects/{proj_name}_{today}.geojson'

### Set up token and API URL

In [3]:
# Set up token access
auth_path = '../secrets.yaml'
with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [4]:
# TerraMatch API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # Use for testing queries
prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" # Use to pull data for analysis

## Pull Polygons from TM API

In [5]:
results = api.pull_wrapper(prod_url, headers, proj_id, outfile=tm_api_pull_results_file)

Pulling Projects:   0%|          | 0/1 [00:00<?, ?project/s]

Pulling Projects: 100%|██████████| 1/1 [00:05<00:00,  5.08s/project]

Results saved to ../data/individual_projects/tm_api_response_prod_justdiggit_2025-06-12.json


In [6]:
# Load the saved JSON file
with open(tm_api_pull_results_file, 'r') as file:
    project_results = json.load(file)

In [9]:
# Clean the csv and transform it into a dataframe
## Identifies and converts invalid plantstart and plantend dates to NaT
## Saves one copy of the polygon features csv to the terrafund-portfolio-analysis repo and one to the maxar-tools repo
clean_api = clean.process_tm_api_results(project_results,
                                         min_valid_plantstart = '2020-01-01',
                                         outfile1 = polygon_features_file,
                                         outfile2 = None)

Number of rows missing a 'plantstart' date: 0/113
Number of rows missing a 'plantend' date: 113/113
⚠️ Total rows missing start and end plant date: 0
⚠️ Total projects with at least 1 polygon missing 'plantstart': 0
⚠️ Total polygons missing 'plantstart': 0
There are 0 projects with ALL polygons missing plantstart.
Projects with SOME polygons missing plantstart: 0


/home/darby/github_repos/terrafund-portfolio-analyses/notebooks/../src/process_tm_api_results.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  affected_rows.loc[is_feb_29] = non_leap_years
/home/darby/github_repos/terrafund-portfolio-analyses/notebooks/../src/process_tm_api_results.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  affected_rows.loc[is_feb_29] = non_leap_years


## Save as a GeoJSON

In [10]:
# Load in csv as dataframe
df = pd.read_csv(polygon_features_file)

In [11]:
geo.df_to_geojson(df, geometry_col='geometry', output_path=geojson_output)

Exported 113 features to ../data/individual_projects/justdiggit_2025-06-12.geojson
